In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor,AdaBoostRegressor, GradientBoostingRegressor

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error, precision_score, recall_score, classification_report, confusion_matrix, f1_score
from sklearn.utils import resample

In [2]:
pd.set_option('display.max_columns', None)

# CARGAS DE TABLAS

In [3]:
tabla1 =  pd.read_csv("C:/Users/Javi/Documents/bootcamp/proyectos/machine_learning/student_data.csv", sep=';')
tabla1

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,reason,guardian,traveltime,studytime,failures,schoolsup,famsup,paid,activities,nursery,higher,internet,romantic,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,course,mother,2,2,0,yes,no,no,no,yes,yes,no,no,4,3,4,1,1,3,6,5,6,6
1,GP,F,17,U,GT3,T,1,1,at_home,other,course,father,1,2,0,no,yes,no,no,no,yes,yes,no,5,3,3,1,1,3,4,5,5,6
2,GP,F,15,U,LE3,T,1,1,at_home,other,other,mother,1,2,3,yes,no,yes,no,yes,yes,yes,no,4,3,2,2,3,3,10,7,8,10
3,GP,F,15,U,GT3,T,4,2,health,services,home,mother,1,3,0,no,yes,yes,yes,yes,yes,yes,yes,3,2,2,1,1,5,2,15,14,15
4,GP,F,16,U,GT3,T,3,3,other,other,home,father,1,2,0,no,yes,yes,no,yes,yes,no,no,4,3,2,1,2,5,4,6,10,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
390,MS,M,20,U,LE3,A,2,2,services,services,course,other,1,2,2,no,yes,yes,no,yes,yes,no,no,5,5,4,4,5,4,11,9,9,9
391,MS,M,17,U,LE3,T,3,1,services,services,course,mother,2,1,0,no,no,no,no,no,yes,yes,no,2,4,5,3,4,2,3,14,16,16
392,MS,M,21,R,GT3,T,1,1,other,other,course,other,1,1,3,no,no,no,no,no,yes,no,no,5,5,3,3,3,3,3,10,8,7
393,MS,M,18,R,LE3,T,3,2,services,other,course,mother,3,1,0,no,no,no,no,no,yes,yes,no,4,4,1,3,4,5,0,11,12,10


In [4]:
print("Valores únicos de la columna Mjob:", tabla1['Mjob'].unique())
print("Valores únicos de la columna Fjob:", tabla1['Fjob'].unique())
print("Valores únicos de la columna reason:", tabla1['reason'].unique())
print("Valores únicos de la columna guardian:", tabla1['guardian'].unique())

Valores únicos de la columna Mjob: ['at_home' 'health' 'other' 'services' 'teacher']
Valores únicos de la columna Fjob: ['teacher' 'other' 'services' 'health' 'at_home']
Valores únicos de la columna reason: ['course' 'other' 'home' 'reputation']
Valores únicos de la columna guardian: ['mother' 'father' 'other']


In [5]:
# Contar los valores únicos de las columnas de tabla1
print("Valores únicos de las columnas de tabla1:")
print(tabla1.nunique())

Valores únicos de las columnas de tabla1:
school         2
sex            2
age            8
address        2
famsize        2
Pstatus        2
Medu           5
Fedu           5
Mjob           5
Fjob           5
reason         4
guardian       3
traveltime     4
studytime      4
failures       4
schoolsup      2
famsup         2
paid           2
activities     2
nursery        2
higher         2
internet       2
romantic       2
famrel         5
freetime       5
goout          5
Dalc           5
Walc           5
health         5
absences      34
G1            17
G2            17
G3            18
dtype: int64


In [6]:
for column in tabla1.columns:
    print(f"\nRecuento de valores únicos en la columna {column}:")
    print(tabla1[column].value_counts())


Recuento de valores únicos en la columna school:
school
GP    349
MS     46
Name: count, dtype: int64

Recuento de valores únicos en la columna sex:
sex
F    208
M    187
Name: count, dtype: int64

Recuento de valores únicos en la columna age:
age
16    104
17     98
18     82
15     82
19     24
20      3
22      1
21      1
Name: count, dtype: int64

Recuento de valores únicos en la columna address:
address
U    307
R     88
Name: count, dtype: int64

Recuento de valores únicos en la columna famsize:
famsize
GT3    281
LE3    114
Name: count, dtype: int64

Recuento de valores únicos en la columna Pstatus:
Pstatus
T    354
A     41
Name: count, dtype: int64

Recuento de valores únicos en la columna Medu:
Medu
4    131
2    103
3     99
1     59
0      3
Name: count, dtype: int64

Recuento de valores únicos en la columna Fedu:
Fedu
2    115
3    100
4     96
1     82
0      2
Name: count, dtype: int64

Recuento de valores únicos en la columna Mjob:
Mjob
other       141
services    103

2 sexo - sexo del alumno (binario: «F» - mujer o «M» - hombre)
3 edad - edad del estudiante (numérico: de 15 a 22 años)
4 address - tipo de domicilio del estudiante (binario: «U» - urbano o «R» - rural)
5 famsize - tamaño de la familia (binario: «LE3» - menor o igual a 3 o «GT3» - mayor de 3)
6 Pstatus - estado de convivencia de los padres (binario: «T» - viven juntos o «A» - separados)
7 Medu - educación de la madre (numérico: 0 - ninguna, 1 - educación primaria (4º grado), 2 - 5º a 9º grado, 3 - educación secundaria o 4 - educación superior)
8 Fedu - educación del padre (numérico: 0 - ninguna, 1 - educación primaria (4º grado), 2 - 5º a 9º grado, 3 - educación secundaria o 4 - educación superior)
9 Mjob - trabajo de la madre (nominal: 'maestra', relacionado con la atención sanitaria, 'servicios' civiles (por ejemplo, administrativo o policía), 'en_casa' u 'otro')
10 Fjob - trabajo del padre (nominal: profesor', 'sanitario', 'servicios' civiles (por ejemplo, administrativo o policía), 'en_casa' u 'otro')
11 razón - razón para elegir este centro (nominal: cerca de «casa», «reputación» del centro, preferencia de «curso» u «otro»)
12 tutor - tutor del alumno (nominal: «madre», «padre» u «otro»)
13 traveltime - tiempo de viaje de casa al colegio (numérico: 1 - <15 min., 2 - 15 a 30 min., 3 - 30 min. a 1 hora, o 4 - >1 hora)
14 tiempo de estudio - tiempo de estudio semanal (numérico: 1 - <2 horas, 2 - 2 a 5 horas, 3 - 5 a 10 horas, o 4 - >10 horas)
15 fracasos - número de fracasos en las clases anteriores (numérico: n si 1<=n<3, si no 4)
16 schoolsup - apoyo educativo adicional (binario: sí o no)
17 famsup - apoyo educativo familiar (binario: sí o no)
18 paid - clases pagadas extra dentro de la asignatura del curso (Matemáticas o Portugués) (binario: sí o no)
19 actividades - actividades extraescolares (binario: sí o no)
20 guardería - asistió a la guardería (binario: sí o no)
21 superior - desea cursar estudios superiores (binario: sí o no)
22 internet - acceso a internet en casa (binario: sí o no)
23 romántico - con una relación romántica (binario: sí o no)
24 famrel - calidad de las relaciones familiares (numérico: de 1 - muy malas a 5 - excelentes)
25 freetime - tiempo libre después de la escuela (numérico: de 1 - muy bajo a 5 - muy alto)
26 goout - salir con amigos (numérico: de 1 - muy bajo a 5 - muy alto)
27 Dalc - consumo de alcohol en días laborables (numérico: de 1 - muy bajo a 5 - muy alto)
28 Walc - consumo de alcohol en fin de semana (numérico: de 1 - muy bajo a 5 - muy alto)
29 salud - estado de salud actual (numérico: de 1 - muy malo a 5 - muy bueno)
30 faltas - número de faltas escolares (numérico: de 0 a 93)




In [7]:
tabla2 =  pd.read_csv("C:/Users/Javi/Documents/bootcamp/proyectos/machine_learning/academic_performance_analytics_complete.csv", sep=',')
tabla2

,Timestamp,Student_ID,Age,Gender,Ethnicity,SES,Location,Enrollment_Status,GPA,Attendance_Rate,Study_Hours_per_Week,Extracurricular_Participation,Course_Load,Major_Field_of_Study,Previous_Academic_Performance,Course_Type,Course_Difficulty,Instructor_Rating,Class_Size,Sentiment_Score,Feedback_Length,Access_to_Learning_Materials,Internet_Accessibility,Counseling_Sessions_Attended,Tutoring_Hours,Assignment_Scores,Midterm_Scores,Final_Exam_Scores,Learning_Style_Compatibility,Career_Alignment_Indicator,Library_Usage_Frequency,Study_Group_Participation,Resource_Access_Score,Learning_Material_Satisfaction,Peer_Interaction_Score,Academic_Support_Utilization,Stress_Indicator_Score,Assignment_Completion_Rate,Learning_Satisfaction_Level,Academic_Performance_Category,Sentiment_Classification,Requirement_Fulfillment_Status,Course_Satisfaction_Level,Engagement_Level
0,2018-07-14 06:00:00,1678,25,0,White,Low,"Texas, Dallas",0,2.283531,0.9,17.647264,0,6,Arts,60.237666,Core,Hard,3.776889,30,0.733300,484,0,1,1,0.000000,83.235872,56.684179,74.283938,0,1,3,0,0.696702,1,0.341722,0,0.346726,0.7,Medium,High,Neutral,Partially Met,Neutral,Active
1,2018-09-27 03:00:00,3106,19,1,White,Low,"Texas, Dallas",0,2.787184,0.6,7.304669,0,5,Engineering,71.919002,Core,Hard,3.708644,30,0.179273,450,0,0,0,0.515937,70.477437,63.487994,69.581913,0,1,3,1,0.655516,0,0.665257,0,0.957973,0.8,Medium,Medium,Negative,Partially Met,Neutral,Active
2,2022-04-25 00:00:00,4350,22,1,African American,Low,"Texas, Dallas",0,2.689566,0.8,12.816062,1,5,Engineering,79.622769,Elective,Easy,4.453540,30,0.357695,496,0,0,0,5.293700,78.254544,81.371728,71.698227,0,0,1,0,0.666346,1,0.501590,0,0.431574,0.7,Low,Medium,Neutral,Fully Met,Satisfied,Moderate
3,2024-02-24 10:00:00,2890,20,0,White,Medium,"Texas, Dallas",0,2.486635,0.6,19.797873,1,4,Arts,82.419630,Elective,Moderate,3.558563,30,0.532631,162,0,0,0,0.000000,90.783972,66.462219,70.034328,1,0,0,0,0.646857,0,0.665658,0,0.633358,0.8,High,High,Neutral,Partially Met,Unsatisfied,Active
4,2020-10-09 23:00:00,4019,25,1,White,Medium,"Texas, Dallas",0,3.258069,1.0,14.457067,0,5,Engineering,69.389492,Advanced,Easy,2.715564,30,0.714675,234,1,1,0,0.830865,65.559076,92.394189,55.446190,0,1,0,0,0.546623,0,0.555273,0,0.581689,0.8,Medium,Medium,Positive,Partially Met,Neutral,Active
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61340,2019-10-29 18:00:00,3885,26,1,White,Low,"Texas, Dallas",0,2.149150,0.7,24.606399,0,4,Arts,78.469013,Elective,Moderate,3.320002,10,0.571314,210,0,0,0,0.000000,79.638439,48.430788,80.654766,0,1,0,1,0.678753,1,0.481441,0,0.627292,0.9,High,Low,Positive,Fully Met,Satisfied,Moderate
61341,2020-09-11 18:00:00,2972,29,1,White,High,"Texas, Dallas",1,2.452148,0.9,13.969722,0,5,Arts,75.114187,Elective,Moderate,4.084590,10,0.491516,101,0,0,0,2.968496,71.845174,70.337474,84.596145,0,1,3,0,0.553857,1,0.621769,0,0.990375,0.8,Medium,High,Neutral,Partially Met,Neutral,Active
61342,2018-11-14 12:00:00,3397,19,1,White,Low,"Texas, Dallas",1,2.372740,0.9,19.750728,0,4,Science,67.443383,Elective,Hard,3.832986,30,0.454625,158,0,0,0,0.000000,69.577117,69.073187,59.010887,0,0,3,0,0.632214,0,0.498190,0,0.469423,0.8,High,Medium,Positive,Partially Met,Satisfied,Active
61343,2023-03-15 22:00:00,4617,25,0,White,High,"Texas, Dallas",0,2.654484,0.8,15.507835,0,4,Law,67.280996,Core,Hard,3.144849,20,0.650554,495,0,0,2,0.872598,65.974002,57.328627,72.791444,1,0,5,1,0.824990,0,0.420456,1,0.633345,0.9,High,Medium,Neutral,Partially Met,Satisfied,Inactive


1 Timestamp Fecha y hora en que se registraron los datos, por horas.
2 Student_ID Identificador único asignado a cada alumno del conjunto de datos.
3 Edad Edad del alumno en el momento de la recogida de datos.
4 Sexo El sexo del alumno (codificado como valores binarios o categóricos).
5 Origen étnico Origen étnico del alumno, basado en los datos demográficos disponibles.
6 SES Indicador del estatus socioeconómico que refleja la procedencia del alumno.
7 Ubicación Ubicación geográfica donde se recogieron los datos, concretamente en el área de Dallas-Fort Worth.
8 Enrollment_Status Estado que indica si el estudiante está matriculado a tiempo completo o a tiempo parcial.
9 GPA Promedio de calificaciones, que representa el rendimiento académico del estudiante.
10 Attendance_Rate Tasa de asistencia a clase, expresada en porcentaje.
11 Study_Hours_per_Week Número de horas que el alumno dedica a estudiar cada semana.
12 Extracurricular_Participation Puntuación que indica el nivel de participación en actividades extracurriculares.
13 Course_Load Número de asignaturas que cursa un alumno durante un periodo determinado.
14 Previous_Academic_Performance Indicador histórico del rendimiento académico del alumno.
15 Course_Type Tipo de curso (por ejemplo, clase, laboratorio, seminario).
16 Calificación_del_instructor Calificación que refleja la satisfacción del alumno con la enseñanza del instructor.
17 Compatibilidad_Estilo_de_Aprendizaje Puntuación que indica en qué medida el estilo de aprendizaje preferido del alumno se ajusta al formato del curso.
18 Indicador de alineación profesional Mide la alineación entre el contenido del curso y los objetivos profesionales del estudiante.
19 Frecuencia de uso de la biblioteca Frecuencia con la que el estudiante accede a la biblioteca o a los recursos de aprendizaje en línea.
20 Study_Group_Participation Participación en grupos de estudio o actividades de aprendizaje colaborativo.
21 Resource_Access_Score Indicador del acceso del alumno a los recursos académicos.
22 Peer_Interaction_Score Medida de la interacción del alumno con sus compañeros.
23 Puntuación del indicador de estrés Puntuación que refleja el nivel de estrés declarado por el alumno.
... ... ...
43 Learning_Satisfaction_Level Indicador de la satisfacción del estudiante con su experiencia de aprendizaje.

In [8]:
tabla3 =  pd.read_csv("C:/Users/Javi/Documents/bootcamp/proyectos/machine_learning/student_prediction.csv", sep=',')
tabla3

,STUDENTID,AGE,GENDER,HS_TYPE,SCHOLARSHIP,WORK,ACTIVITY,PARTNER,SALARY,TRANSPORT,LIVING,MOTHER_EDU,FATHER_EDU,#_SIBLINGS,KIDS,MOTHER_JOB,FATHER_JOB,STUDY_HRS,READ_FREQ,READ_FREQ_SCI,ATTEND_DEPT,IMPACT,ATTEND,PREP_STUDY,PREP_EXAM,NOTES,LISTENS,LIKES_DISCUSS,CLASSROOM,CUML_GPA,EXP_GPA,COURSE ID,GRADE
0,STUDENT1,2,2,3,3,1,2,2,1,1,1,1,2,3,1,2,5,3,2,2,1,1,1,1,1,3,2,1,2,1,1,1,1
1,STUDENT2,2,2,3,3,1,2,2,1,1,1,2,3,2,1,2,1,2,2,2,1,1,1,1,1,3,2,3,2,2,3,1,1
2,STUDENT3,2,2,2,3,2,2,2,2,4,2,2,2,2,1,2,1,2,1,2,1,1,1,1,1,2,2,1,1,2,2,1,1
3,STUDENT4,1,1,1,3,1,2,1,2,1,2,1,2,5,1,2,1,3,1,2,1,1,1,1,2,3,2,2,1,3,2,1,1
4,STUDENT5,2,2,1,3,2,2,1,3,1,4,3,3,2,1,2,4,2,1,1,1,1,1,2,1,2,2,2,1,2,2,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140,STUDENT141,2,1,2,3,1,1,2,1,1,2,1,2,2,2,2,4,3,3,2,1,1,1,1,1,2,1,2,1,3,3,9,5
141,STUDENT142,1,1,2,4,2,2,2,1,4,2,1,1,5,1,2,1,3,2,2,2,1,2,1,1,3,2,2,1,5,3,9,5
142,STUDENT143,1,1,1,4,2,2,2,1,1,1,3,4,4,1,2,4,2,2,2,1,1,1,1,1,3,3,2,1,4,3,9,1
143,STUDENT144,2,1,2,4,1,1,1,5,2,3,4,4,1,1,3,3,2,2,1,1,1,1,2,1,2,1,2,1,5,3,9,4


1- Edad del estudiante (1: 18-21, 2: 22-25, 3: más de 26)
2- Sexo (1: femenino, 2: masculino)
3- Tipo de escuela secundaria: (1: privada, 2: pública, 3: otra)
4- Tipo de beca: (1: ninguna, 2: 25%, 3: 50%, 4: 75%, 5: completa)
5- Trabajo adicional: (1: Sí, 2: No)
6- Actividad artística o deportiva regular: (1: Sí, 2: No)
7- Tiene pareja: (1: Sí, 2: No)
8- Salario total si lo tiene (1: 135-200 USD, 2: 201-270 USD, 3: 271-340 USD, 4: 341-410 USD, 5: más de 410 USD)
9- Transporte a la universidad: (1: Autobús, 2: Coche privado/taxi, 3: Bicicleta, 4: Otro)
10- Tipo de alojamiento en Chipre: (1: alquiler, 2: dormitorio, 3: con familia, 4: otro)
11- Estudios de la madre: (1: primaria, 2: secundaria, 3: bachillerato, 4: universidad, 5: máster, 6: doctorado).
12- Estudios del padre: (1: primaria, 2: secundaria, 3: bachillerato, 4: universidad, 5: máster, 6: doctorado).
13- Número de hermanas/hermanos (si lo tiene): (1: 1, 2:, 2, 3: 3, 4: 4, 5: 5 o más)
14- Situación de los padres: (1: casados, 2: divorciados, 3: fallecidos - uno de ellos o ambos) ***Listado como «Hijos»...woops
15- Profesión de la madre: (1: jubilada, 2: ama de casa, 3: funcionaria, 4: empleada del sector privado, 5: autónoma, 6: otra)
16- Profesión del padre: (1: jubilado, 2: funcionario, 3: empleado del sector privado, 4: autónomo, 5: otros)
17- Horas de estudio semanales: (1: ninguna, 2: <5 horas, 3: 6-10 horas, 4: 11-20 horas, 5: más de 20 horas)
18- Frecuencia de lectura (libros/revistas no científicas): (1: Ninguna, 2: A veces, 3: A menudo)
19- Frecuencia de lectura (libros/revistas científicas): (1: Ninguna, 2: A veces, 3: A menudo)
20- Asistencia a los seminarios/conferencias relacionados con el departamento: (1: Sí, 2: No)
21- Impacto de sus proyectos/actividades en su éxito: (1: positivo, 2: negativo, 3: neutro)
22- Asistencia a clase (1: siempre, 2: a veces, 3: nunca)
23- Preparación de los exámenes parciales 1: (1: solo, 2: con amigos, 3: no aplicable)
24- Preparación a los exámenes parciales 2: (1: fecha más próxima al examen, 2: regularmente durante el semestre, 3: nunca)
25- Tomar apuntes en clase: (1: nunca, 2: a veces, 3: siempre)
26- Escuchar en clase: (1: nunca, 2: a veces, 3: siempre)
27- El debate mejora mi interés y mi éxito en el curso: (1: nunca, 2: a veces, 3: siempre)
28- Flip-classroom: (1: no útil, 2: útil, 3: no aplicable)
29- Nota media acumulada en el último semestre (/4,00): (1: <2,00, 2: 2,00-2,49, 3: 2,50-2,99, 4: 3,00-3,49, 5: superior a 3,49)
30- Promedio acumulativo esperado en la graduación (/4.00): (1: <2,00, 2: 2,00-2,49, 3: 2,50-2,99, 4: 3,00-3,49, 5: superior a 3,49)
31- ID del curso
32- SALIDA Calificación (0: Suspenso, 1: DD, 2: DC, 3: CC, 4: CB, 5: BB, 6: BA, 7: AA)

In [9]:
tabla4 =  pd.read_csv("C:/Users/Javi/Documents/bootcamp/proyectos/machine_learning/StudentsPerformance.csv", sep=',')
tabla4

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75
...,...,...,...,...,...,...,...,...
995,female,group E,master's degree,standard,completed,88,99,95
996,male,group C,high school,free/reduced,none,62,55,55
997,female,group C,high school,free/reduced,completed,59,71,65
998,female,group D,some college,standard,completed,68,78,77


# TIPOS DE DATOS

In [10]:
# Tipos de datos de las columnas de tabla1
print("Tipos de datos de tabla1:")
print(tabla1.dtypes)

# Tipos de datos de las columnas de tabla2
print("\nTipos de datos de tabla2:")
print(tabla2.dtypes)

# Tipos de datos de las columnas de tabla3
print("\nTipos de datos de tabla3:")
print(tabla3.dtypes)


Tipos de datos de tabla1:
school        object
sex           object
age            int64
address       object
famsize       object
Pstatus       object
Medu           int64
Fedu           int64
Mjob          object
Fjob          object
reason        object
guardian      object
traveltime     int64
studytime      int64
failures       int64
schoolsup     object
famsup        object
paid          object
activities    object
nursery       object
higher        object
internet      object
romantic      object
famrel         int64
freetime       int64
goout          int64
Dalc           int64
Walc           int64
health         int64
absences       int64
G1             int64
G2             int64
G3             int64
dtype: object

Tipos de datos de tabla2:
Timestamp                          object
Student_ID                          int64
Age                                 int64
Gender                              int64
Ethnicity                          object
SES                           

In [11]:
# Mostrar todas las columnas de tabla1
print("Columnas de tabla1:")
print(tabla1.head())

# Mostrar todas las columnas de tabla2
print("\nColumnas de tabla2:")
print(tabla2.head())

# Mostrar todas las columnas de tabla3
print("\nColumnas de tabla3:")
print(tabla3.head())

# Mostrar todas las columnas de tabla4
print("\nColumnas de tabla4:")
print(tabla4.head())

Columnas de tabla1:
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

   reason guardian  traveltime  studytime  failures schoolsup famsup paid  \
0  course   mother           2          2         0       yes     no   no   
1  course   father           1          2         0        no    yes   no   
2   other   mother           1          2         3       yes     no  yes   
3    home   mother           1          3         0        no    yes  yes   
4    home   father           1          2         0        no    yes  yes   

  activities nursery higher internet romantic  famrel  freet

In [12]:
tabla3.head(1)

,STUDENTID,AGE,GENDER,HS_TYPE,SCHOLARSHIP,WORK,ACTIVITY,PARTNER,SALARY,TRANSPORT,LIVING,MOTHER_EDU,FATHER_EDU,#_SIBLINGS,KIDS,MOTHER_JOB,FATHER_JOB,STUDY_HRS,READ_FREQ,READ_FREQ_SCI,ATTEND_DEPT,IMPACT,ATTEND,PREP_STUDY,PREP_EXAM,NOTES,LISTENS,LIKES_DISCUSS,CLASSROOM,CUML_GPA,EXP_GPA,COURSE ID,GRADE
0,STUDENT1,2,2,3,3,1,2,2,1,1,1,1,2,3,1,2,5,3,2,2,1,1,1,1,1,3,2,1,2,1,1,1,1


# TABLA 1

In [13]:
tabla1["pareja"] = tabla1["romantic"]
tabla1["divorciados"] = tabla1["Pstatus"]
tabla1["fam_numerosa"] = tabla1["famsize"]
tabla1["genero"] = tabla1["sex"]
tabla1["pareja"] = tabla1["romantic"]

tabla1['media'] = tabla1[['G1', 'G2', 'G3']].mean(axis=1).round(2)
tabla1['aprobado'] = (tabla1['media'] >= 5).astype(int)
tabla1['apoyo_edu'] = ((tabla1['schoolsup'] == 'yes') | (tabla1['famsup'] == 'yes') | (tabla1['paid'] == 'yes')).astype(int)
tabla1['divorciados'] = ((tabla1['divorciados'] == 'A') | (tabla1['famsup'] == 'yes') | (tabla1['paid'] == 'yes')).astype(int)

tabla1 = tabla1.drop(['school', "G1", "G2", "G3","higher", "nursery", "failures", "reason", "guardian", "traveltime", "schoolsup", "famsup", "paid", "sex", "pstatus"], axis=1)
tabla1.columns = [col.lower() for col in tabla1.columns]

tabla1.head()

KeyError: "['pstatus'] not found in axis"

In [27]:
tabla1 = tabla1.drop(["pstatus"], axis=1)
tabla1.head()

,age,address,famsize,medu,fedu,mjob,fjob,studytime,activities,internet,romantic,famrel,freetime,goout,dalc,walc,health,absences,pareja,divorciados,fam_numerosa,genero,media,aprobado,apoyo_edu
0,18,U,GT3,4,4,at_home,teacher,2,no,no,no,4,3,4,1,1,3,6,no,1,GT3,F,5.67,1,1
1,17,U,GT3,1,1,at_home,other,2,no,yes,no,5,3,3,1,1,3,4,no,1,GT3,F,5.33,1,1
2,15,U,LE3,1,1,at_home,other,2,no,yes,no,4,3,2,2,3,3,10,no,1,LE3,F,8.33,1,1
3,15,U,GT3,4,2,health,services,3,yes,yes,yes,3,2,2,1,1,5,2,yes,1,GT3,F,14.67,1,1
4,16,U,GT3,3,3,other,other,2,no,no,no,4,3,2,1,2,5,4,no,1,GT3,F,8.67,1,1


In [ ]:
'Mjob': {'at_home': 1, 'health': 2, 'other': 3, 'services': 4, 'teacher': 5},
'Fjob': {'at_home': 1, 'health': 2, 'other': 3, 'services': 4, 'teacher': 5},
'reason': {'course': 1, 'home': 2, 'other': 3, 'reputation': 4},
'guardian': {'mother': 1, 'father': 2, 'other': 3},

In [36]:
# Crear un diccionario de mapeo para cada columna
mappings = {
    'fam_numerosa': {'GT3': 1, 'LE3': 0},
    'divorciados': {'A': 1, 'T': 0},
    'activities': {'yes': 1, 'no': 0},
    'internet': {'yes': 1, 'no': 0},
    'pareja': {'yes': 1, 'no': 0}
}

# Aplicar el mapeo a cada columna
for column, mapping in mappings.items():
    new_column = column + '_num'
    tabla1[new_column] = tabla1[column].map(mapping)
    cols = tabla1.columns.tolist()
    col_index = cols.index(column)
    cols.insert(col_index + 1, cols.pop(cols.index(new_column)))
    tabla1 = tabla1[cols]

tabla1

In [ ]:
tabla_final = tabla1["genero", "edad", "fam_numerosa", "pareja", "media", "aprobado", "apoyo_edu", "", "divorciado", "Pstatus"]

# TABLA 2

In [ ]:
tabla2.head(1)


,Timestamp,Student_ID,Age,Gender,Ethnicity,SES,Location,Enrollment_Status,GPA,Attendance_Rate,Study_Hours_per_Week,Extracurricular_Participation,Course_Load,Major_Field_of_Study,Previous_Academic_Performance,Course_Type,Course_Difficulty,Instructor_Rating,Class_Size,Sentiment_Score,Feedback_Length,Access_to_Learning_Materials,Internet_Accessibility,Counseling_Sessions_Attended,Tutoring_Hours,Assignment_Scores,Midterm_Scores,Final_Exam_Scores,Learning_Style_Compatibility,Career_Alignment_Indicator,Library_Usage_Frequency,Study_Group_Participation,Resource_Access_Score,Learning_Material_Satisfaction,Peer_Interaction_Score,Academic_Support_Utilization,Stress_Indicator_Score,Assignment_Completion_Rate,Learning_Satisfaction_Level,Academic_Performance_Category,Sentiment_Classification,Requirement_Fulfillment_Status,Course_Satisfaction_Level,Engagement_Level
0,2018-07-14 06:00:00,1678,25,0,White,Low,"Texas, Dallas",0,2.283531,0.9,17.647264,0,6,Arts,60.237666,Core,Hard,3.776889,30,0.7333,484,0,1,1,0.0,83.235872,56.684179,74.283938,0,1,3,0,0.696702,1,0.341722,0,0.346726,0.7,Medium,High,Neutral,Partially Met,Neutral,Active


In [29]:
tabla2 = tabla2.drop(['Timestamp', "Student_ID", "Enrollment_Status", "Course_Type", "Course_Difficulty", "Enrollment_Status", "Course_Type", "Course_Difficulty", "Course_Load"], axis=1)

In [30]:
tabla2.head(1)


,Age,Gender,Ethnicity,SES,Location,GPA,Attendance_Rate,Study_Hours_per_Week,Extracurricular_Participation,Course_Load,Major_Field_of_Study,Previous_Academic_Performance,Instructor_Rating,Class_Size,Sentiment_Score,Feedback_Length,Access_to_Learning_Materials,Internet_Accessibility,Counseling_Sessions_Attended,Tutoring_Hours,Assignment_Scores,Midterm_Scores,Final_Exam_Scores,Learning_Style_Compatibility,Career_Alignment_Indicator,Library_Usage_Frequency,Study_Group_Participation,Resource_Access_Score,Learning_Material_Satisfaction,Peer_Interaction_Score,Academic_Support_Utilization,Stress_Indicator_Score,Assignment_Completion_Rate,Learning_Satisfaction_Level,Academic_Performance_Category,Sentiment_Classification,Requirement_Fulfillment_Status,Course_Satisfaction_Level,Engagement_Level
0,25,0,White,Low,"Texas, Dallas",2.283531,0.9,17.647264,0,6,Arts,60.237666,3.776889,30,0.7333,484,0,1,1,0.0,83.235872,56.684179,74.283938,0,1,3,0,0.696702,1,0.341722,0,0.346726,0.7,Medium,High,Neutral,Partially Met,Neutral,Active


In [ ]:
tabla2_ok = tabla2[["Age",'Gender', "Ethnicity", "SES", "GPA", "Final_Exam_Scores" "Study_Hours_per_Week", "Extracurricular_Participation",'Access_to_Learning_Materials', "Internet_Accessibility", "Counseling_Sessions_Attended", "Library_Usage_Frequency", "Study_Hours_per_Week"]]

In [5]:
# Identificar las columnas que contienen letras
columns_to_map = tabla2.select_dtypes(include=['object']).columns

# Crear un diccionario de mapeo para cada columna
mappings = {}
for column in columns_to_map:
    unique_values = tabla2[column].unique()
    if len(unique_values) == 2:
        mappings[column] = {unique_values[0]: 0, unique_values[1]: 1}
    else:
        mappings[column] = {value: idx + 1 for idx, value in enumerate(sorted(unique_values))}

# Aplicar el mapeo a cada columna y reordenar las columnas
for column, mapping in mappings.items():
    new_column = column + '_num'
    tabla2[new_column] = tabla2[column].map(mapping)
    cols = tabla2.columns.tolist()
    col_index = cols.index(column)
    cols.insert(col_index + 1, cols.pop(cols.index(new_column)))
    tabla2 = tabla2[cols]


In [6]:
tabla2


In [16]:

tabla2.columns = [col.lower() for col in tabla2.columns]
tabla2

# TABLA 3

In [19]:
# Identificar las columnas categóricas
categorical_columns = tabla3.select_dtypes(include=['object']).columns

# Convertir las columnas categóricas en one hot encoding
tabla3_one_hot = pd.get_dummies(tabla3, columns=categorical_columns)

tabla3_one_hot

In [8]:
# Identificar las columnas que contienen letras
columns_to_map = tabla3.select_dtypes(include=['object']).columns

# Crear un diccionario de mapeo para cada columna
mappings = {}
for column in columns_to_map:
    unique_values = tabla3[column].unique()
    if len(unique_values) == 2:
        mappings[column] = {unique_values[0]: 0, unique_values[1]: 1}
    else:
        mappings[column] = {value: idx + 1 for idx, value in enumerate(sorted(unique_values))}

# Aplicar el mapeo a cada columna y reordenar las columnas
for column, mapping in mappings.items():
    new_column = column + '_num'
    tabla3[new_column] = tabla3[column].map(mapping)
    cols = tabla3.columns.tolist()
    col_index = cols.index(column)
    cols.insert(col_index + 1, cols.pop(cols.index(new_column)))
    tabla3 = tabla3[cols]

tabla3

In [15]:

tabla3.columns = [col.lower() for col in tabla3.columns]
tabla3

# TABLA 4

In [12]:
tabla4 = tabla4.drop(columns=['STUDENTID'])
tabla4

In [14]:

tabla4.columns = [col.lower() for col in tabla4.columns]
tabla4

In [17]:
# Nombres de las columnas de tabla1
columnas_tabla1 = tabla1.columns.tolist()
print("Columnas de tabla1:", columnas_tabla1)

# Nombres de las columnas de tabla2
columnas_tabla2 = tabla2.columns.tolist()
print("Columnas de tabla2:", columnas_tabla2)

# Nombres de las columnas de tabla3
columnas_tabla3 = tabla3.columns.tolist()
print("Columnas de tabla3:", columnas_tabla3)

# Nombres de las columnas de tabla4
columnas_tabla4 = tabla4.columns.tolist()
print("Columnas de tabla4:", columnas_tabla4)

In [18]:
# Nombres de las columnas de tabla1
print("Columnas de tabla1:")
for col in columnas_tabla1:
    print(col)

# Nombres de las columnas de tabla2
print("\nColumnas de tabla2:")
for col in columnas_tabla2:
    print(col)

# Nombres de las columnas de tabla3
print("\nColumnas de tabla3:")
for col in columnas_tabla3:
    print(col)

# Nombres de las columnas de tabla4
print("\nColumnas de tabla4:")
for col in columnas_tabla4:
    print(col)